In [1]:
import time
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
project_id = 'price-smart-initialpricing'
!gcloud config set project {project_id}
from google.cloud import storage
from google.colab import auth
!gcloud auth login --update-adc

Updated property [core/project].
Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=glIh5oSuLK1M7mhV68THNVJQ9Cckz2&prompt=consent&access_type=offline&code_challenge=_V9v6fghacQHyA9bae1xyeGhUpP-WeAuJRL07zBKlNM&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk7Clp1GsVe5YFfFOyhWNSZqLjNgXcSJvtXDnP-6B_5ixEcdKgH9PYrd5shzlVDZow

Application default credentials (ADC) were updated.

You are now logged in as [shashank.vats@impactanalytics.co].
Your current project

In [9]:
from google.cloud import bigquery

client = bigquery.Client(project='price-smart-initialpricing')

query_gp = """
          SELECT DISTINCT gfe_material_nbr AS gfe_material_nbr 
          FROM `price-smart-initialpricing.RL_incremental.gfe_images_mapping` 
          WHERE ((asset_gsutil_url IS NOT NULL) OR (asset_gsutil_url <> "") OR (asset_gsutil_url <> " ")) 
            AND gsutil_url LIKE '%invalid_image%' 
            AND gfe_material_nbr IN (
              SELECT DISTINCT material_number 
            FROM `price-smart-initialpricing.RL_incremental.similarity_mapping_input`
  )

"""

query_results_gp = client.query(query_gp)

df_global = query_results_gp.to_dataframe()

In [10]:
df_global

,gfe_material_nbr
0,833820199001
1,312892565001
2,323799823004
3,311884247008
4,322882970001
...,...
8247,211882312002
8248,323902097001
8249,211867488001
8250,417609989001


There are **8252** out of **80985** materials that have no images in their service url but have valid asset_url.

In [11]:
from google.cloud import bigquery

client = bigquery.Client(project='price-smart-initialpricing')

query_gp = """
          SELECT DISTINCT gfe_material_nbr AS gfe_material_nbr 
          FROM `price-smart-initialpricing.RL_incremental.gfe_images_mapping` 
          WHERE ((asset_gsutil_url IS NULL) OR (asset_gsutil_url = "") OR (asset_gsutil_url = " ")) 
            AND gsutil_url not LIKE '%invalid_image%' 
            AND gfe_material_nbr IN (
              SELECT DISTINCT material_number 
            FROM `price-smart-initialpricing.RL_incremental.similarity_mapping_input`
  )

"""

query_results_gp = client.query(query_gp)

df_global = query_results_gp.to_dataframe()

In [12]:
df_global

,gfe_material_nbr
0,282737734001
1,321703663001
2,405727128001
3,427793183001
4,710675188001
...,...
15191,782733253001
15192,211709351001
15193,680556920001
15194,710646709018


There are **15196** out of **80985** materials where the asset_ url is invalid but we get good image through service
